In [ ]:
import compyute as cp
from compyute.preprocessing.text import CharacterTokenizer

In [ ]:
import requests

response = requests.get("https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt")
data = response.text

chars = sorted(list(set(data)))
tokenizer = CharacterTokenizer()
tokenizer.vocab = {i: c for i, c in enumerate(chars)}
tokenizer.ivocab = {c: i for i, c in enumerate(chars)}

In [ ]:
vocab_size = tokenizer.vocab_size
block_size = 256
embed_dims = 384
device = cp.cuda

In [ ]:
from transformer.gpt import GPTTransformer
from transformer.attention_funcs import get_causal_mask

mask = get_causal_mask((block_size, block_size))

model = GPTTransformer(
    n_embeddings=vocab_size,
    embedding_dim=embed_dims,
    ffwd_channels=4 * embed_dims,
    n_heads=6,
    n_blocks=6,
    max_seq_len=block_size,
    mask=mask,
)
model.to_device(device)
state_dict = cp.load("experiments/shakespeare/transformer_shakespeare_5_5000.cp")
model.load_state_dict(state_dict["model"])

In [ ]:
context = "Second Murderer:"
print(context, end="")
context = tokenizer.encode(context)  # encode context
context = cp.tensor(context, cp.cuda, cp.int32).view((1, -1))  # insert batch dim

for _ in range(500):
    logits = model(context)[0, -1].to_cpu()  # get logits of last token
    probs = cp.nn.functional.softmax(logits)  # convert to probs
    topk_probs, topk_indices = cp.topk(probs, 50)  # get top 50 probs
    topk_probs /= topk_probs.sum()  # normalize probs
    index = cp.random.multinomial(x=50, p=topk_probs, shape=(1,))  # sample
    index = topk_indices[index]  # get token id
    char = tokenizer.decode(index.to_list())
    print(char, end="")
    index = index.view((1, 1))
    context = cp.append(context[:, 1:], values=index, dim=1).to_int()  # append to previous context